In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import when, col, sum, count
from itertools import combinations
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StructType, StructField, LongType

import pandas as pd

spark = SparkSession.builder.appName("Yelp").getOrCreate()

In [ ]:
yelp_rest = spark.read.csv("/content/la_restaurants_cleaned.csv", header = True, inferSchema = True)
yelp_rest = yelp_rest.withColumnRenamed("bid", "rest_bid")
yelp_rest = yelp_rest.withColumnRenamed("stars", "rest_stars")
yelp_rest = yelp_rest.withColumnRenamed("review_count", "rest_review_count")
yelp_rest = yelp_rest.withColumnRenamed("city", "rest_city")
yelp_rest.show(5)

+--------+--------------------+-----------+--------------------+-------+-------------+--------------+----------+-----------------+
|rest_bid|           rest_name|  rest_city|        rest_address|zipcode|rest_latitude|rest_longitude|rest_stars|rest_review_count|
+--------+--------------------+-----------+--------------------+-------+-------------+--------------+----------+-----------------+
|   97600|Fireman Mike's Ca...|New Orleans|  203 W Harrison Ave|  70124|      30.0059|      -90.1173|       4.5|               14|
|  106952|              Nomiya|New Orleans|    4226 Magazine St|  70115|      29.9208|      -90.0998|       4.5|              258|
|  121230|The Melting Pot -...|New Orleans|1820 Saint Charle...|  70130|      29.9357|       -90.078|       3.5|              128|
|    3229|Chubbie's Fried C...|New Orleans|4850 General Meye...|  70131|      29.9321|      -90.0031|       4.0|               70|
|    5964|    Key's Food Store|New Orleans|  2600 Louisiana Ave|  70115|      29.93

In [ ]:
yelp_rest_rids = spark.read.csv("/content/rest_rids_bids.csv", header = True, inferSchema = True)
yelp_rest_rids.show(5)

+------+--------------------+-----------+---------------+-----+-----------+--------+---------+-----+------------+-------------+--------------------+
|   bid|                name|       city|        address|state|postal_code|latitude|longitude|stars|review_count|category_name|           review_id|
+------+--------------------+-----------+---------------+-----+-----------+--------+---------+-----+------------+-------------+--------------------+
|125000|The Original Pier...|New Orleans|440 Chartres St|   LA|      70130| 29.9557| -90.0652|  4.0|        1886|   Sandwiches|dzJULPmc02gQnrHx5...|
|125000|The Original Pier...|New Orleans|440 Chartres St|   LA|      70130| 29.9557| -90.0652|  4.0|        1886|   Sandwiches|WjhljnWeYna2ZhYEj...|
|125000|The Original Pier...|New Orleans|440 Chartres St|   LA|      70130| 29.9557| -90.0652|  4.0|        1886|   Sandwiches|_zI-H_UmC-J3hermC...|
|125000|The Original Pier...|New Orleans|440 Chartres St|   LA|      70130| 29.9557| -90.0652|  4.0|      

In [ ]:
merge_la_rests_reviews = yelp_rest.join(yelp_rest_rids, yelp_rest["rest_bid"] == yelp_rest_rids["bid"], "left")
merge_la_rests_reviews = merge_la_rests_reviews.drop("bid", "name", "city", "address", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "category_name" )
merge_la_rests_reviews.show(5)

+--------+----------------+-----------+------------------+-------+-------------+--------------+----------+-----------------+--------------------+
|rest_bid|       rest_name|  rest_city|      rest_address|zipcode|rest_latitude|rest_longitude|rest_stars|rest_review_count|           review_id|
+--------+----------------+-----------+------------------+-------+-------------+--------------+----------+-----------------+--------------------+
|    5964|Key's Food Store|New Orleans|2600 Louisiana Ave|  70115|      29.9373|      -90.0961|       4.0|               13|ERX4SUAzz_aSsicVD...|
|    5964|Key's Food Store|New Orleans|2600 Louisiana Ave|  70115|      29.9373|      -90.0961|       4.0|               13|ms4rSc0V3P2CuPgN1...|
|    5964|Key's Food Store|New Orleans|2600 Louisiana Ave|  70115|      29.9373|      -90.0961|       4.0|               13|fNWsfkvTzQQjBmble...|
|    5964|Key's Food Store|New Orleans|2600 Louisiana Ave|  70115|      29.9373|      -90.0961|       4.0|               13|

In [ ]:
yelp_reviews = spark.read.csv("/content/drive/MyDrive/Poster project/yelp_academic_dataset_review_withtext.csv", header = False, inferSchema = True)
yelp_reviews.show()

+--------------------+--------------------+
|                 _c0|                 _c1|
+--------------------+--------------------+
|KU_O5udG6zpxOg-Vc...|If you decide to ...|
|BiTunyQ73aT9WBnpR...|I've taken a lot ...|
|saUsX_uimxRlCVr67...|Family diner. Had...|
|AqPFMleE6RsU23_au...|Wow!  Yummy, diff...|
|Sx8TMOWLNuJBWer-0...|Cute interior and...|
|JrIxlS1TzJ-iCu79u...|I am a long term ...|
|6AxgBCNX_PNTOxmbR...|Loved this tour! ...|
|_ZeMknuYdlQcUqng_...|Amazingly amazing...|
|ZKvDG2sBvHVdF5oBN...|This easter inste...|
|pUycOfUwM8vqX7KjR...|Had a party of 6 ...|
|rGQRf8UafX7OTlMNN...|My experience wit...|
|l3Wk_mvAog6XANIuG...|Locals recommende...|
|XW_LfMv0fV21l9c6x...|Love going here f...|
|8JFGBuHMoiNDyfcxu...|Good food--loved ...|
|UBp0zWyH60Hmw6Fsa...|The bun makes the...|
|OAhBYw8IQ6wlfw1ow...|Great place for b...|
|oyaMhzBSwfGgemSGu...|Tremendous servic...|
|LnGZB0fjfgeVDVz5I...|The hubby and I h...|
|u2vzZaOqJ2feRshaa...|I go to blow bar ...|
|Xs8Z8lmKkosqW5mw_...|My absolut

In [ ]:
yelp_reviews = yelp_reviews.withColumnRenamed("_c0", "rid")
yelp_reviews = yelp_reviews.withColumnRenamed("_c1", "review_text")
yelp_reviews.show(5)

+--------------------+--------------------+
|                 rid|         review_text|
+--------------------+--------------------+
|KU_O5udG6zpxOg-Vc...|If you decide to ...|
|BiTunyQ73aT9WBnpR...|I've taken a lot ...|
|saUsX_uimxRlCVr67...|Family diner. Had...|
|AqPFMleE6RsU23_au...|Wow!  Yummy, diff...|
|Sx8TMOWLNuJBWer-0...|Cute interior and...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
merged_la = merge_la_rests_reviews.join(yelp_reviews, merge_la_rests_reviews["review_id"] == yelp_reviews["rid"], "inner")
merged_la.show(5)

+--------+--------------------+-----------+---------------+-------+-------------+--------------+----------+-----------------+--------------------+--------------------+--------------------+
|rest_bid|           rest_name|  rest_city|   rest_address|zipcode|rest_latitude|rest_longitude|rest_stars|rest_review_count|           review_id|                 rid|         review_text|
+--------+--------------------+-----------+---------------+-------+-------------+--------------+----------+-----------------+--------------------+--------------------+--------------------+
|  125000|The Original Pier...|New Orleans|440 Chartres St|  70130|      29.9557|      -90.0652|       4.0|             1886|-5xjAlN0_tDmkv8pl...|-5xjAlN0_tDmkv8pl...|We had a wonderfu...|
|  125000|The Original Pier...|New Orleans|440 Chartres St|  70130|      29.9557|      -90.0652|       4.0|             1886|-5xjAlN0_tDmkv8pl...|-5xjAlN0_tDmkv8pl...|We had a wonderfu...|
|  125000|The Original Pier...|New Orleans|440 Chartres

In [ ]:
merged_la = merged_la.drop("review_id", "rid")
merged_la.coalesce(1).write.csv("merged_la.csv", header = True)

In [ ]:
tn_rests = spark.read.csv("/content/tn_restaurants_cleaned.csv", header = True, inferSchema = True)
tn_rests = tn_rests.withColumnRenamed("bid", "rest_bid")
tn_rests = tn_rests.withColumnRenamed("stars", "rest_stars")
tn_rests = tn_rests.withColumnRenamed("review_count", "rest_review_count")
tn_rests = tn_rests.withColumnRenamed("city", "rest_city")
tn_rests.show(5)

+--------+--------------------+---------+--------------------+-------+-------------+--------------+----------+-----------------+
|rest_bid|           rest_name|rest_city|        rest_address|zipcode|rest_latitude|rest_longitude|rest_stars|rest_review_count|
+--------+--------------------+---------+--------------------+-------+-------------+--------------+----------+-----------------+
|  136581|       Newk's Eatery|Nashville|      2714 W End Ave|  37203|      36.1454|      -86.8123|       3.5|               96|
|  137954|   Papa John's Pizza|Nashville|5814 Nolensville ...|  37211|      36.0436|      -86.7125|       1.5|               40|
|  145697| San Antonio Taco Co|Nashville|     208 Commerce St|  37201|      36.1631|      -86.7765|       2.5|               19|
|   62946|Asihi Asian Cuisi...|Nashville|7648 Hwy 70 S, Ste 1|  37221|      36.0797|      -86.9524|       3.0|               89|
|   68260|E+Rose Wellness C...|Nashville|       211 3rd Ave S|  37201|       36.159|      -86.774

In [ ]:
tn_rids = spark.read.csv("/content/tn_rest_rids.csv", header = True, inferSchema = True)
tn_rids.show(5)

+------+-------------+---------+--------------+-----+------------+--------------------+
|   bid|         name|     city|       address|stars|review_count|           review_id|
+------+-------------+---------+--------------+-----+------------+--------------------+
|136581|Newk's Eatery|Nashville|2714 W End Ave|  3.5|          96|6c7UkBaNpZJIggJZZ...|
|136581|Newk's Eatery|Nashville|2714 W End Ave|  3.5|          96|W1AbKQcivgZvmQz6x...|
|136581|Newk's Eatery|Nashville|2714 W End Ave|  3.5|          96|k6MRRrHxNrNuxLLq0...|
|136581|Newk's Eatery|Nashville|2714 W End Ave|  3.5|          96|qYJzWe4GF0CGeDctM...|
|136581|Newk's Eatery|Nashville|2714 W End Ave|  3.5|          96|NCQNZwonUC2KSmgmn...|
+------+-------------+---------+--------------+-----+------------+--------------------+
only showing top 5 rows



In [ ]:
merge_tn_rests_reviews = tn_rests.join(tn_rids, tn_rests["rest_bid"] == tn_rids["bid"], "left")
merge_tn_rests_reviews = merge_tn_rests_reviews.drop("bid", "name", "city", "address", "rest_latitude", "rest_longitude", "stars", "review_count" )
merge_tn_rests_reviews.show(5)

+--------+--------------------+---------+-------------+-------+----------+-----------------+--------------------+
|rest_bid|           rest_name|rest_city| rest_address|zipcode|rest_stars|rest_review_count|           review_id|
+--------+--------------------+---------+-------------+-------+----------+-----------------+--------------------+
|   68260|E+Rose Wellness C...|Nashville|211 3rd Ave S|  37201|       4.5|               87|N9iA_t_aAWX85pmPa...|
|   68260|E+Rose Wellness C...|Nashville|211 3rd Ave S|  37201|       4.5|               87|yUf9T8qkXBNuFvy9x...|
|   68260|E+Rose Wellness C...|Nashville|211 3rd Ave S|  37201|       4.5|               87|KBaLwIEEP7w2D7Rfs...|
|   68260|E+Rose Wellness C...|Nashville|211 3rd Ave S|  37201|       4.5|               87|izndcY9yXu60URXdl...|
|   68260|E+Rose Wellness C...|Nashville|211 3rd Ave S|  37201|       4.5|               87|kWx-mU1WxPKqPADQf...|
+--------+--------------------+---------+-------------+-------+----------+--------------

In [ ]:
yelp_reviews = spark.read.csv("/content/drive/MyDrive/Poster project/yelp_academic_dataset_review_withtext.csv", header = False, inferSchema = True)
yelp_reviews = yelp_reviews.withColumnRenamed("_c0", "rid")
yelp_reviews = yelp_reviews.withColumnRenamed("_c1", "review_text")
yelp_reviews.show(5)

+--------------------+--------------------+
|                 rid|         review_text|
+--------------------+--------------------+
|KU_O5udG6zpxOg-Vc...|If you decide to ...|
|BiTunyQ73aT9WBnpR...|I've taken a lot ...|
|saUsX_uimxRlCVr67...|Family diner. Had...|
|AqPFMleE6RsU23_au...|Wow!  Yummy, diff...|
|Sx8TMOWLNuJBWer-0...|Cute interior and...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
merged_tn = merge_tn_rests_reviews.join(yelp_reviews, merge_tn_rests_reviews["review_id"] == yelp_reviews["rid"], "inner")
merged_tn.show(5)

+--------+----------+---------+--------------------+-------+----------+-----------------+--------------------+--------------------+--------------------+
|rest_bid| rest_name|rest_city|        rest_address|zipcode|rest_stars|rest_review_count|           review_id|                 rid|         review_text|
+--------+----------+---------+--------------------+-------+----------+-----------------+--------------------+--------------------+--------------------+
|   19081|China King|Nashville|6601 Sugar Valley Dr|  37211|       4.0|               39|-1j8Th523DOHr_-OG...|-1j8Th523DOHr_-OG...|This is our go to...|
|   19081|China King|Nashville|6601 Sugar Valley Dr|  37211|       4.0|               39|-1j8Th523DOHr_-OG...|-1j8Th523DOHr_-OG...|This is our go to...|
|   19081|China King|Nashville|6601 Sugar Valley Dr|  37211|       4.0|               39|-1j8Th523DOHr_-OG...|-1j8Th523DOHr_-OG...|This is our go to...|
|   19081|China King|Nashville|6601 Sugar Valley Dr|  37211|       4.0|           

In [ ]:
merged_tn = merged_tn.drop("review_id", "rid")
merged_tn.coalesce(1).write.csv("merged_tn.csv", header = True)